# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [2]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3RU1AG/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3RU1AG/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [3]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [4]:
word2vec.wv.most_similar('loyal', topn=10)

[('nuclear', 0.9947394132614136),
 ('rhonda', 0.9931631088256836),
 ('standing', 0.9928145408630371),
 ('jr', 0.9926080703735352),
 ('messages', 0.9925793409347534),
 ('kinnear', 0.9924481511116028),
 ('competition', 0.9922499060630798),
 ('ultra', 0.9920902252197266),
 ('fictional', 0.9916597604751587),
 ('southern', 0.9915211796760559)]

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [ ]:
# YOUR CODE HERE

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [ ]:
# YOUR CODE HERE

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [5]:
word2vec.wv.similar_by_vector('loyal', topn=10)

[('nuclear', 0.9947394132614136),
 ('rhonda', 0.9931631088256836),
 ('standing', 0.9928145408630371),
 ('jr', 0.9926080703735352),
 ('messages', 0.9925793409347534),
 ('kinnear', 0.9924481511116028),
 ('competition', 0.9922499060630798),
 ('ultra', 0.9920902252197266),
 ('fictional', 0.9916597604751587),
 ('southern', 0.9915211796760559)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [6]:
word2vec.wv['good'] - word2vec.wv['bad']

array([-0.23808208, -0.7526066 ,  0.1568008 , -0.19111106,  0.5227575 ,
       -0.29385805,  0.3912732 ,  0.21794549,  0.2660277 ,  0.17034245,
        0.22256872,  0.04423094,  0.03064632,  0.42478526,  0.47194052,
       -0.36244756,  0.04382905,  0.03657329,  0.0796033 ,  0.2727133 ,
       -0.51638585,  0.27138382,  0.3869527 , -0.06284356,  0.30851012,
        0.09636495,  0.152125  , -0.13578135, -0.03628653,  0.16506642,
       -0.08946529,  0.5211306 , -0.07133608,  0.09417745,  0.00484103,
       -0.2691673 ,  0.31766516, -0.02934301, -0.75329804,  0.31635207,
       -0.2965638 ,  0.10447744, -0.12972367,  0.33285046, -0.19491278,
       -0.06385779,  0.16158079, -0.14595854, -0.09525806, -0.20439851,
       -0.2550301 ,  0.07774529, -0.8002226 , -0.10433812, -0.08738005,
        0.0969355 , -0.39952683,  0.02302172,  0.26700598,  0.34494364,
        0.3447407 , -0.15534508, -0.23619014,  0.18929619, -0.10962564,
        0.51476157,  0.1309678 ,  0.18441708, -0.23727608, -0.18

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [7]:
res = word2vec.wv['good'] - word2vec.wv['bad'] + word2vec.wv['stupid']
res

array([-2.27543324e-01, -3.27050090e-01, -2.68544167e-01, -5.52484035e-01,
        1.53927028e-01,  4.35066223e-01,  3.94537330e-01, -7.04795122e-02,
       -5.98546147e-01,  2.48098582e-01,  2.07848936e-01,  4.40605313e-01,
        1.60817634e-02,  6.63990676e-02, -1.43121779e-01, -2.14175463e-01,
       -1.61121786e-01, -1.53446630e-01, -2.10376456e-02, -1.37194097e-01,
       -4.98903215e-01,  6.27298236e-01,  4.44159001e-01,  7.03713745e-02,
        1.26761451e-01, -1.27300546e-01, -3.30085576e-01, -1.70470446e-01,
        1.89487636e-01,  6.11386076e-02, -2.87554897e-02,  3.66318464e-01,
        3.77750993e-02, -9.66545045e-02, -4.32579100e-01, -7.82607645e-02,
        1.49418309e-01,  2.73418963e-01, -4.21932787e-01,  4.13622111e-01,
       -1.63128600e-01,  3.65727335e-01,  4.93355691e-02,  9.52232540e-01,
       -3.71003211e-01, -3.61944079e-01,  2.47309834e-01, -1.04015596e-01,
       -3.02067339e-01,  3.13313186e-01,  5.90832829e-02,  1.92873329e-01,
       -4.49981511e-01, -

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [8]:
word2vec.wv.similar_by_vector(res)

[('such', 0.8242133855819702),
 ('decent', 0.813824474811554),
 ('nice', 0.8015741109848022),
 ('deed', 0.7958776950836182),
 ('potential', 0.7900720834732056),
 ('rotten', 0.7871522903442383),
 ('cute', 0.7832990884780884),
 ('gunslinger', 0.7826628684997559),
 ('consistently', 0.7825126647949219),
 ('known', 0.7794952988624573)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [ ]:
# YOUR CODE HERE

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [9]:
word2vec_2 = Word2Vec(sentences=X_train, size=20)

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [10]:
len(word2vec.wv.index2word), len(word2vec_2.wv.index2word)

(8006, 8006)

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [11]:
word2vec_3 = Word2Vec(sentences=X_train, size=20, min_count=7)
len(word2vec_3.wv.index2word)

6064

In [12]:
word2vec_4 = Word2Vec(sentences=X_train, size=20, min_count=1)
len(word2vec_4.wv.index2word)

30419

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [13]:
word2vec_5 = Word2Vec(sentences=X_train, size=20, window=2, min_count=9)
len(word2vec_4.wv.index2word)

30419

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [24]:
type(word2vec)

gensim.models.word2vec.Word2Vec

In [31]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']


def embed_sentence(word2vec, sentence):
    embed_list = []
    for word in sentence:
        if word in word2vec.wv:
           embed_list.append(word2vec.wv[word])
    return np.array(embed_list)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [16]:
len(X_train)

2500

In [32]:

def embedding(word2vec, sentences):
    embedded_sentence_list = []
    for sentence in sentences:
        embedded_sentence_list.append(
            embed_sentence(word2vec, sentence))
        
    return embedded_sentence_list

    
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)




In [36]:
len(X_train)

2500

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [37]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)